## Week 5 Lecture 2 - MCMC

McElreath's lecture for today: https://www.youtube.com/watch?v=Qqz5AJjyugM

An R/Stan repo of code is available here: https://vincentarelbundock.github.io/rethinking2/

An excellent port to Python/PyMC Code is available here: https://github.com/dustinstansbury/statistical-rethinking-2023

You are encouraged to work through both of these versions to re-enforce what we're doing in class.

In [1]:
# Import python packages
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy as sp 
import random as rd
import pdb
import pymc as pm
import arviz as az
from matplotlib import pyplot as plt


# Helper functions
def stdize(x):
    return (x-np.mean(x))/np.std(x)


def indexall(L):
    poo = []
    for p in L:
        if not p in poo:
            poo.append(p)
    Ix = np.array([poo.index(p) for p in L])
    return poo,Ix

## Markov Chains

The miracle of modern computation is that it has made practical the use of Markov chain Monte Carlo (MCMC) for numerical approximation of all the various derivatives inherent in calculating the posterior of a Bayesian statistical model. What is remarkable is how simple the most basic algorithm is: the Metropolis algorithm. 

![](maniac.jpg)

[Nicholas Metropolis](https://en.wikipedia.org/wiki/Nicholas_Metropolis) was a remarkable physicist on the Manhattan Project who worked out the compression and ingnition of materials within the first atomic bomb. In fact, a deep irony of Bayesian statistics, now used to help heal the world through fields such as medicine and conservation, [owes it's widespread use to the most destructive project of all time](http://library.lanl.gov/cgi-bin/getfile?07-15.pdf). You can listen to an interview with him at the [Voices of the Manhattan Project](https://www.manhattanprojectvoices.org/oral-histories/nicholas-metropolis-interview) archive. Metropolis also set up a joke at the start of the Woody Allen film [Husbands and Wives](https://www.youtube.com/watch?v=_ojZxFjlFV0).

The Metropolis algorithm was published in a 1953 article [Equation of State Calculations by Fast Computing Machines](https://people.umass.edu/bvs/The_1953_paper.pdf) that set out to understand the physical properties of fluids. So what did Metropolis do? Well he developed the MANIAC computer where the calculations were done and, along with two wife-husband teams, Arianna & Marshall Rosenbluth, and Augusta & Edward Teller he developed an algothrim that cracked the code on modern Bayesian implementation. Rumor has it that the Rosenbluth's actually did the idea and implementation (the heavy lifting), while Metropolis just stuck his name out front (jerk move). Whatever happened, a good MCMC algorithm is required to do one thing, namely to **explore the parameter space in proportion to it's posterior probability**. That's it. But doing so requires two key things, (1) **proposal** system and (2) some criteria for **acceptance**. 

The algorithm from the 1953 paper used the following equation to set out a proposed move from a current value ($x_{1}$) to ($x_{2}$):

$$
x_{1}\rightarrow x_{1}+\alpha \xi_{1}
$$

with $\alpha$ being a maximum displacement allowed at each step and $\xi_{1}$ being a random number between -1 and 1 (which is symmetric). They generated random numbers on [MANIAC I](https://en.wikipedia.org/wiki/MANIAC_I), using the [middle square method](https://en.wikipedia.org/wiki/Middle-square_method). You can read the complete computer manual here: http://www.bitsavers.org/pdf/lanl/LA-1725_The_MANIAC_Jul54.pdf, with the random number generator on p240. (Incidentally, [uniform distributions](https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)) are the basis of a key form of random number generation for any probabilty distribution, [rejection sampling](https://en.wikipedia.org/wiki/Rejection_sampling)).

Let's follow along at home, using our built-in uniform random number generator:

In [ ]:
# Initial value
x1 = 5

# Arbitrary maximum distance
α = 15

# Initial proposal
x2p = x1+α*np.random.uniform(-1,1)
x2p

The next step connects us back to Shannon and entropy, whereby the energy of the system is used as a basis acceptance for the new value $x_2$. In Metropolis *et al.* (1953) they calculated the potential energy of their system of fluids according to 

$$
E = \frac{1}{2}\sum^{N}_{i=1}\sum^{N}_{j=1}V(d_{ij}) \enspace for \enspace i\neq j
$$

which accounts for the relative positions of particles in a field according to the energy potential between molecules (V) and the distance between them $d_{ij}$. In our calculation we need some way to calculate the 'potential energy' of our initial value $x_1$ and our new value $x_2$. Because we don't have particles with an energy calculation to minimize, let's assert that what we're trying to estimate is equal to $\pi=3.14159$. So we can now use our algorithm to determine if our $x_2$ proposal is further or closer to $\pi$ than our curent $x_1$ value - in other words we can minimize to get some directionality going. In our case, we can use

$$
E_i = |\pi-x_i|
$$

to get our 'energies' for each value:

In [ ]:
# Energy for x1
x1pe = abs(np.pi-x1)
x1pe

In [ ]:
# Energy for x2
x2pe = abs(np.pi-x2p)
x2pe

With the energies in hand, Metropolis *et al.* then calculated the net energy change in the system $\Delta E$ and applied the following acceptance criteria:

1. if $\Delta E$<0, accept $x_2$
2. otherwise accept $x_2$ with probability $e^{-\Delta E/kT}$
    
where $k$ is the [Boltzman constant](https://en.wikipedia.org/wiki/Boltzmann_constant) and T is temperature. In our case we can assume k and T are constant. So implementing this 

In [ ]:
# Delta energy
ΔE = x2pe-x1pe
ΔE

And we can apply the acceptance criteria, using

In [ ]:
if (ΔE<0):
    p_accept = 1
else:
    p_accept = np.exp(-ΔE)
p_accept

And then take a Bernoulli sample to either accept or reject $x_2$

In [ ]:
x2 = np.random.binomial(1, p_accept)
x2 

Putting this all together into an algorithmic scheme, we can create our markov chain-based Metropolis algorithm for, say, 1000 samples:

In [ ]:
# Initial value
x = [5]
# Arbitrary maximum distance
α = 15

for i in range(0,10000):
    # Proposal
    proposal = x[i-1]+α*np.random.uniform(-1,1)
    # Change in energy
    ΔE = abs(np.pi-proposal)-abs(np.pi-x[i-1])
    # Acceptance algorithm
    if (ΔE<0):
        p_accept = 1
    else:
        p_accept = np.exp(-ΔE)
    accept = np.random.binomial(1, p_accept)
    # Markov step i
    x += [x[i-1]*(1-accept)+proposal*accept]

In [ ]:
plt.plot(x)
plt.title('Your first wiggly caterpillar')
plt.savefig('wiggly.jpg', dpi=300);

In [ ]:
az.plot_density(np.array(x))
plt.axvline(np.pi)
plt.savefig('mcmcpi.jpg', dpi=300);

That this works, is completely insane. I'm in continual disbelief that it does. Nontheless...

# Why MCMC?

Many (most) times, you can't integrate analytically (do math) so we need to solve these things numerically, through simulation. This includes hierarhical models, phylogentic analysis, and spatial models for example. In a huge range of  cases optimization fails and in a big range of cases approximations (e.g. Laplace) fail too. 

## MCMC algorithms in history

- Metropolis (1953) - [Equation of State Calculations by Fast Computing Machines](https://aip.scitation.org/doi/10.1063/1.1699114)
- Metropolis-Hastings (1970) - [Monte Carlo Sampling Methods Using Markov Chains and Their Applications ](https://www.jstor.org/stable/2334940?seq=1)
- Gibbs sampling (1984) - [Stochastic Relaxation, Gibbs Distributions, and the Bayesian Restoration of Images](https://ieeexplore.ieee.org/document/4767596)
- Hamiltonian Monte Carlo (1987) - [Hybrid Monte Carlo](https://www.sciencedirect.com/science/article/abs/pii/037026938791197X)

The first three are 'quess and check' methods (propose=guess, accept=check); everyting in the future lies in gradients.

Take a look at each of these algorithms (and more) in practice, here: http://chi-feng.github.io/mcmc-demo/

# Hamiltonian Monte Carlo

Ok, so we've been told the future is in gradients - what the heck are those? Gradients are the derivatives (or slopes) for all dimensions of a problem. If you have a normal posterior this gives us the steepness of the slope at some combination of $\mu$ and $\sigma$, conditional on the data. So let's do this for the example outlines from p273 to 278 un the book; the first is to simulate some data, which I'll do from $x_i \sim N(2.3,1)$ and  $y_i \sim N(-1.7,1)$:


In [ ]:
# Number of samples
nsamp = 100

# Simulate x
xtrue = 2.3
x = np.random.normal(xtrue,1,nsamp)
# Simulate y
ytrue = -1.7
y = np.random.normal(ytrue,1,nsamp)

# Plot 'em
plt.scatter(x,y)
plt.xlabel('N(2.3,1)', fontsize=15)
plt.ylabel('N(-1.7,1)', fontsize=15)
plt.savefig('rando.jpg', dpi=300);

The next step is outline our statistical model:

$$
x_i \sim N(\mu_x,1)\\
y_i \sim N(\mu_y,1)\\
\mu_x \sim N(0,0.5)\\
\mu_y \sim N(0,0.5)
$$

which is just describing normal distributions for x and y respectively. To do HMC we need to get a few things in order, namely:

1. The log-probability of the data and parameters
2. The gradient
3. Number of leapfrog steps
4. Step size


First the **log-probability of the data and parameters**, which we've seen before, this time can be written as:

$$
\sum_{i}log p(x_i|\mu_x,1)+\sum_{i}log p(y_i|\mu_y,1)+log p(\mu_x|0,0.5)+log p(\mu_y|0,0.5)
$$

These are the log-probabilities for each of the shorhand notations of our model above, with the first two terms being the data likelihoods for $x$ and $y$ and the second two terms being their priors. Note that the sigmas in the data likelihood are fixed at 1 to make this all a bit simpler. So let's calculate this first thing:

In [ ]:
# log-probability of the data, model, and priors at the initial values
sum(sp.stats.norm.logpdf(x,0,1))+sum(sp.stats.norm.logpdf(y,0,1))+sp.stats.norm.logpdf(0,0,0.5)+sp.stats.norm.logpdf(0,0,0.5)

So given our prior means of zero, the log-probability is that. But we need to be able to do this algorithmically, calculating this same thing at each step in our sampler. To do this we write a function

In [ ]:
# log-probability function
def lp(x,y,mu,a=0,b=0.5,k=0,d=.5):
    return -(sum(sp.stats.norm.logpdf(x,mu[1],1))+sum(sp.stats.norm.logpdf(y,mu[0],1))+sp.stats.norm.logpdf(mu[1],k,d)+sp.stats.norm.logpdf(mu[0],a,b))

The second bit is the **gradient**, meaning the partial derivatives to get at the slopes. By design we're using Normal didstributions because Normals are easy to work with. The derivative of the logarithm for $N(\mu,\sigma)$ with respect to $\mu$ is

$$
\frac{\delta log N(x|\mu,\sigma)}{\delta \mu} = \frac{x-\mu^2}{\sigma^2}
$$

And because the derivative of a sum is a sum of derivatives, we can get the derivative for the prior and likelihood for the x data in our model given

$$
\frac{\delta log N(x|\mu_x,\sigma)}{\delta \mu_x}+\frac{\delta log N(\mu_x|0,0.5)}{\delta x} = \sum_{i}\frac{x_i-\mu_x}{1^2}+\frac{0-\mu_x}{0.5^2}
$$

and for y, given
$$
\frac{\delta log N(y|\mu_y,\sigma)}{\delta \mu_y}+\frac{\delta log N(\mu_y|0,0.5)}{\delta y} = \sum_{i}\frac{y_i-\mu_y}{1^2}+\frac{0-\mu_y}{0.5^2}
$$

To program this, we need another function

In [ ]:
# Gradient function
def lpg(x,y,mu,a=0,b=0.5,k=0,d=0.5):
    G1 = sum(y-mu[0])+(a-mu[0])/(b**2)
    G2 = sum(x-mu[1])+(k-mu[1])/(d**2)
    # Return gradients as negative because energy is negative log-probability
    return np.array([-G1, -G2])

The number of **leapfrog steps** ($L$) asks how many points we want to sample along the trajectory of our likelihood and the **step size** ($s$) determines how big each of those *L* steps should be. In practice these things are chosen for us, but for the sake of this example, we'll use 11 leapfrog steps ($L=11$), with a step size of 0.03 ($s=0.03$). 

Ok with all these things in place, we can build our Hamiltonian algorithm. First the Hamiltoninan function for setting things in motion

In [ ]:
def HMC(lp, lpg, current_q, x, y, s=0.03, L=11):
    # Store current parameters
    q = current_q
    # Random flick on momentum from N(0,1)
    p = np.random.normal(0,1,len(q))
    # Store current momentum
    current_p = p
    # Make a half-step for momentum p
    p = p-s*lpg(x, y, q)/2
    # Create output matrix for parameter trajectory
    qtraj = np.ones((L+1,len(q)))*-999
    # Empty matrix for momentum
    ptraj = np.ones((L+1,len(q)))*-999
    # Append initial values
    qtraj[0,] = current_q
    ptraj[0,] = p
    
    # Alternate full steps for position and momentum
    for i in range(L):
        # move position (parameters) by step size x current gradient
        q = q+s*p
        # Make a full step for momentum unless at last leapfrog step
        if i!=(L-1):
            p = p - s*lpg(x, y, q)
            # Append new momentum
            ptraj[i+1,] = p
        # Append new position
        qtraj[i+1,] = q
    
    # Make a half step for momentum at the end
    p = p - s*lpg(x, y, q)/2
    ptraj[L,] = p
    
    # Negate momentum at the end of the trajectory to make the proposal symmetric
    p = -p
    
    # Evaluate potential and kinetic energies at the start of trajectory
    current_lp = lp(x, y, current_q)
    current_K = sum(current_p**2)/2
    # Total inital energy
    H0 = current_lp+current_K
    
    # Evaluate potential and kinetic energies at the end of trajectory
    proposed_lp = lp(x, y, q)
    proposed_K = sum(p**2)/2
    # Total proposed energy
    H1 = proposed_lp+proposed_K

    # Accept or reject end state
    accept = 0
    if np.random.uniform(0,1)<np.exp(current_lp-proposed_lp+current_K-proposed_K):
        new_q = q
        accept = 1
    else:
        new_q = current_q
    
    return new_q, qtraj, ptraj, accept, H1-H0
    

This HMC function does one Hamiltonian step - to build up a posteior we need to make a Markov chain to store and access the trace over, say, 1000 samples. We can do that here:

In [ ]:
# Number of samples
nsamps = 1000
# Empty matrix to hold traces
trace = np.ones((nsamps+1,2))*-999
# Initialize
trace[0,] = np.array([0.08319271, -0.08801522])
# Iterate
for i in range(nsamps):
    trace[i+1,] = HMC(lp, lpg, trace[i], x, y)[0]


In [ ]:
_, ax = plt.subplots(1,2, figsize=(10,4))

az.plot_density(trace.T[0],ax=ax[0])
m50 = np.median(trace.T[0])
ax[0].axvline(ytrue,c='red')
ax[0].set_title('μ_y='+str(ytrue))

ax[1].plot(trace.T[0])
plt.savefig('hmcy.jpg', dpi=300);

In [ ]:
_, ax = plt.subplots(1,2, figsize=(10,4))

az.plot_density(trace.T[1],ax=ax[0])
m50 = np.median(trace.T[1])
ax[0].axvline(xtrue,c='red')
ax[0].set_title('μ_x='+str(xtrue))

ax[1].plot(trace.T[1])
plt.savefig('hmcx.jpg', dpi=300);

So not too bad - it's worth playing with this code to see how things change with changing the leapfrog and step sizes.